# Transfer learning for NLP classification

In [1]:
from fastai.text import *
import html

In [2]:
PATH = Path('data/aclImdb/')

In [3]:
SOS = 'xbos' # Start/beginning of sentence
FLD = 'xfld' # data field tag

## Standardize format

In [4]:
# Classification path: sentiment analysis
CLASS_PATH = Path('data/imdb_class/')
CLASS_PATH.mkdir(exist_ok=True)

# Language model path
LM_PATH = Path('data/imdb_lm/')
LM_PATH.mkdir(exist_ok=True)

In [5]:
CLASSES = ['neg', 'pos', 'unsup']

In [6]:
def get_texts(path):
    texts, labels = [], []
    for idx, label in enumerate(CLASSES):
        for fname in (path/label).glob('*.*'):
            texts.append(fname.open('r', encoding='utf-8').read())
            labels.append(idx)
    return np.array(texts), np.array(labels)

In [7]:
trn_texts, trn_labels = get_texts(PATH/'train')

In [8]:
val_texts, val_labels = get_texts(PATH/'test')

In [9]:
len(trn_texts), len(val_texts)

(75000, 25000)

### Shuffle the dataset ...

In [10]:
np.random.seed(42)
trn_idx = np.random.permutation(len(trn_texts))
val_idx = np.random.permutation(len(val_texts))

In [11]:
trn_texts = trn_texts[trn_idx]
val_texts = val_texts[val_idx]

trn_labels = trn_labels[trn_idx]
val_labels = val_labels[val_idx]

### ... and create pandas dataframes to store data as csv

In [12]:
column_names = ['labels', 'text']

In [13]:
df_trn = pd.DataFrame({'text':trn_texts, 'labels':trn_labels}, columns=column_names)

In [14]:
df_val = pd.DataFrame({'text':val_texts, 'labels':val_labels}, columns=column_names)

#### csv files for classification
Remove all texts with label `unsup`.

In [15]:
df_trn[df_trn['labels']!=2].to_csv(CLASS_PATH/'train.csv', header=False, index=False)

In [16]:
df_val.to_csv(CLASS_PATH/'test.csv', header=False, index=False)

In [17]:
(CLASS_PATH/'classes.txt').open('w', encoding='utf-8').writelines(f'{o}\n' for o in CLASSES)

In [27]:
df_trn.head()

,labels,text
0,2,In the first and most memorable scene in this ...
1,0,"Why is this film so bad? Well, if being so stu..."
2,1,Ruth Gordon is one of the more sympathetic kil...
3,2,I used to subscribe to Toyfare magazine and on...
4,2,...and I don't mean nuclear!! This movie is pr...


#### csv files for language model
Labels can be ignored, so also `unsup` labeled texts can be used.

In [22]:
trn_texts, val_texts = sklearn.model_selection.train_test_split(np.concatenate([trn_texts, val_texts]), test_size=0.1)

In [23]:
len(trn_texts), len(val_texts)

(90000, 10000)

In [24]:
df_trn = pd.DataFrame({'text':trn_texts, 'labels':[0]*len(trn_texts)}, columns=column_names)

In [25]:
df_val = pd.DataFrame({'text':val_texts, 'labels':[0]*len(val_texts)}, columns=column_names)

In [26]:
df_trn.head()

,labels,text
0,0,"Hoo dawgies, what can I say about this movie t..."
1,0,"Back when cable TV started, there were tons of..."
2,0,"Viewing this film makes you question, no pun i..."
3,0,"no, this is not supposed to be a high budget b..."
4,0,"This is meant to be a thriller, but the audien..."


In [27]:
df_trn.to_csv(LM_PATH/'train.csv', header=False, index=False)
df_val.to_csv(LM_PATH/'test.csv', header=False, index=False)

## Tokenization
Cleanup of text:

1. Replace extra spaces, tab chars, new line chars and other chars with standard tokens
1. Use *spacy* to tokenize the data. Fastai adds multicore functionality to spacy.

*Chunksize:* `pd.read_csv` returns iterator so that we don't have to load the entire dataset into memory at once. 

In [29]:
chunksize = 24000

In [30]:
# Compile a regular expression pattern, returning a pattern object.
new_re = re.compile(r' +')

def cleanup(x):
    '''Many nlp datasets contain strange characters. This function removes them.'''
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>','u_n').replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return new_re.sub(' ', html.unescape(x))

In [31]:
# Start every training/testing example with SOS token so that language model can learn that a new text starts and it might need to forget its state. If there are several text columns, start each of them with {FLD} index so that the language model is able to distinguish between the columns.

def get_texts(df, n_lbls=1):
    labels = df.iloc[:, range(n_lbls)].values.astype(np.int64)
    texts = f'\n{SOS} {FLD} 1 ' + df[n_lbls].astype(str)

    # here {FLD} 1 will be used twice if there is more than 1 text column i think. TODO: check this out
    for i in range(n_lbls+1, len(df.columns)):
        texts += f' {FLD} {i-nlbls} ' + df[i].astype(str)
        
    # cleanup disturbing chars
    texts = list(texts.apply(cleanup).values)
    
    # tokenize using multicore functionality
    tok = Tokenizer().proc_all_mp(partition_by_cores(texts))
    
    return tok, list(labels)

In [32]:
def get_all(df, n_lbls):
    tok, labels = [], []
    # remeber that df is split into pieces of size chunksize
    for i, r in enumerate(df):
        print(i)
        tok_, labels_ = get_texts(r, n_lbls)
        tok += tok_
        labels += labels_
        
    return tok, labels

In [54]:
df_trn = pd.read_csv(LM_PATH/'train.csv', header=None, chunksize=chunksize)

df_val = pd.read_csv(LM_PATH/'test.csv', header=None, chunksize=chunksize)

**Attention:**
If at this point of the notebook you do sth like this, you lose the first chunksize samples: 
`next(iter(df_trn)).head()`
Don't do that!

In [55]:
tok_trn, trn_labels = get_all(df_trn, 1)
tok_val, val_labels = get_all(df_val, 1)

0
1
2
3
0


In [35]:
(LM_PATH/'tmp').mkdir(exist_ok=True)

#### Save tokens

In [36]:
np.save(LM_PATH/'tmp'/'tok_trn.npy', tok_trn)
np.save(LM_PATH/'tmp'/'tok_val.npy', tok_val)

In [37]:
tok_trn = np.load(LM_PATH/'tmp'/'tok_trn.npy')
tok_val = np.load(LM_PATH/'tmp'/'tok_val.npy')

In [38]:
# Counter counts hashable items
frequency = Counter(tok for i in tok_trn for tok in i)

In [39]:
frequency.most_common(25)

[('the', 1208409),
 ('.', 991990),
 (',', 985348),
 ('and', 588114),
 ('a', 583960),
 ('of', 525638),
 ('to', 485093),
 ('is', 393943),
 ('it', 341786),
 ('in', 337659),
 ('i', 307547),
 ('this', 270652),
 ('that', 261453),
 ('"', 236846),
 ("'s", 221476),
 ('-', 188208),
 ('was', 180289),
 ('\n\n', 178920),
 ('as', 165777),
 ('with', 159249),
 ('for', 158941),
 ('movie', 157440),
 ('but', 150728),
 ('film', 144225),
 ('you', 124078)]

**Discard every token that appears less than 2 times. Vocabulary sizes <= 60,000 have been observed to yield good results.**

In [40]:
max_vocabs = 60000
min_freq = 2

In [41]:
itos = [tok for tok, count in frequency.most_common(max_vocabs) if count > min_freq]

In [42]:
itos.insert(0, '_pad_')
itos.insert(0, '_unk_')

**Reverse mapping from string to index:**

In [43]:
stoi = collections.defaultdict(lambda:0, {val:key for key, val in enumerate(itos)})

In [44]:
len(itos)

60002

In [45]:
trn_lm = np.array([[stoi[string] for string in text] for text in tok_trn])
val_lm = np.array([[stoi[string] for string in text] for text in tok_val])

**Save:**

In [46]:
np.save(LM_PATH/'tmp'/'trn_ids.npy', trn_lm)
np.save(LM_PATH/'tmp'/'val_ids.npy', val_lm)

pickle.dump(itos, open(LM_PATH/'tmp'/'itos.pkl', 'wb'))

In [47]:
trn_lm = np.load(LM_PATH/'tmp'/'trn_ids.npy')
val_lm = np.load(LM_PATH/'tmp'/'val_ids.npy')

itos = pickle.load(open(LM_PATH/'tmp'/'itos.pkl', 'rb'))

In [48]:
voc_size = len(itos)
voc_size, len(trn_lm)

(60002, 90000)